In [1]:
!pip install langchain_community
!pip install langchain
!pip install pypdf
!pip install chromadb
!pip install docx2txt

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3960 sha256=63d6fc4285b67cf8ca7495179a7135495675b847bfbb82d6ec77ef8ba921e168
  Stored in directory: /Users/claudioed/Library/Caches/pip/wheels/6f/81/48/001bbc0109c15e18c009eee300022f42d1e070e54f1d00b218
Successfully built docx2txt


In [2]:
from langchain_community.document_loaders import Docx2txtLoader
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
DOC_PATH = "../docs/exercicio-api-management.docx"
CHROMA_PATH = "word" 

loader = Docx2txtLoader(DOC_PATH)
pages = loader.load()

# chunk the content
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(pages)

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# embed the chunks as vectors and load them into the database
db_chroma = Chroma.from_documents(chunks, embeddings, persist_directory=CHROMA_PATH)


/Users/claudioed/development/ai/langchain-intro/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [3]:
query = 'what is goal of this exercise?'

# retrieve context - top 5 most relevant (closests) chunks to the query vector 
# (by default Langchain is using cosine distance metric)
docs_chroma = db_chroma.similarity_search_with_score(query, k=5)

# generate an answer based on given user query and retrieved context information
context_text = "\n\n".join([doc.page_content for doc, _score in docs_chroma])

print(context_text)

Repositorio de apoio:

https://github.com/claudioed/kong-ingress-fiap



Objetivos:

Realizar o deployment do Kong API Gateway em um cluster de Kubernetes.

Configurar o Kong para gerenciar chamadas de API.

Validar a configuração através de uma chamada de API utilizando o curl.

Instruções:

Preparação do Ambiente:

Deployment de um Kong API Gateway em Kubernetes para Gestão de API

Contexto:

A gestão eficaz de APIs é crucial para o desenvolvimento e operação de serviços modernos na nuvem. Neste exercício, você aplicará conceitos de API Management através da implementação de um Kong API Gateway em um ambiente Kubernetes. O Kong API Gateway é uma ferramenta open source poderosa para gerenciar APIs REST e GRPC, proporcionando segurança, controle de tráfego, análise, e muito mais.

Repositorio de apoio:

Adicione o repositório do Helm do Kong e faça a instalação do chart do Kong API Gateway. Configure o Kong para operar em modo DB-less.

Configuração do Gateway:

Após o deployment, conf

In [4]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

# you can use a prompt template
PROMPT_TEMPLATE = """
    Answer the question based only on the following context:
    {context}
    Answer the question based on the above context: {question}.
    Provide a detailed answer.
    Don’t justify your answers.
    Don’t give information not mentioned in the CONTEXT INFORMATION.
    Do not say "according to the context" or "mentioned in the context" or similar.
"""

# load retrieved context and user query in the prompt template
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query)

# call LLM model to generate the answer based on the given context and query
model = ChatOpenAI()
response_text = model.predict(prompt)

print(response_text)

/Users/claudioed/development/ai/langchain-intro/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/Users/claudioed/development/ai/langchain-intro/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


The goal of this exercise is to deploy the Kong API Gateway in a Kubernetes cluster, configure it to manage API calls, and validate the configuration by making an API call using curl. This exercise aims to demonstrate the effective management of APIs using Kong API Gateway in a Kubernetes environment, providing security, traffic control, analysis, and other features for modern cloud services.
